In [2]:
from model import get_models_and_path
models = get_models_and_path()


In [7]:
resFreeze = models[-4]
print(resFreeze[0].__class__.__name__)
print(resFreeze[1])
res = models[-5]
print(res[0].__class__.__name__)
print(res[1])


In [14]:
target = "gene1.0.bias"

for name, param in resFreeze[0].named_parameters():
    if name != target:
        continue
    
    print(name, ": ", param.data)
    
for name, param in res[0].named_parameters():
    if name != target:
        continue
    #print(name)
    print(name, ": ", param.data)
    

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../Training_Data/p009/Preprocessed_STDataset/gene_data.csv")
print(df.head())

In [4]:
print(df["MKI67"])

In [9]:
df = pd.read_csv("../Training_Data/p013/Preprocessed_STDataset/gene_data.csv")
print(df.head())
mk = df["MKI67"]
rb = df["RUBCNL"]

#RUBCNL

In [10]:
print(rb.min())
print(mk.min())


In [11]:
rb.value_counts()

In [12]:
mk.value_counts()


In [18]:
class A():
    def __init__(self, mylist):
        for item in mylist:
            setattr(self, item, item + "xxx")
            print(getattr(self, item))
            print(item)
a = A(["a", "b"])

axxx
a
bxxx
b


In [19]:
a.c = "asb"
print(getattr(a, "b"))
print(a.b)
print(a.c)

bxxx
bxxx
asb


<class '__main__.a'>


In [9]:
import json
json_path = "./test.json"
with open(json_path, "w") as f:
    json_dict = {"type":"vgg13", "random_weights":True, 'gene_list':["abc", "def"], "dropout": True, "pretrained_output_dim":1000}
    json.dump(json_dict, f)
    
with open(json_path) as f:
    d = json.load(f)
    print(d)
    model_type = d["type"]
    print(type(model_type), model_type)
    random_weights = d["random_weights"]
    print(type(random_weights), random_weights)
    
    gene_list = d["gene_list"]
    print(type(gene_list), gene_list)
    
    dropout = d["dropout"]
    print(type(dropout), dropout)
    
    pretrained_output_dim = d["pretrained_output_dim"]
    print(type(pretrained_output_dim), pretrained_output_dim)
    

{'type': 'vgg13', 'random_weights': True, 'gene_list': ['abc', 'def'], 'dropout': True, 'pretrained_output_dim': 1000}
<class 'str'> vgg13
<class 'bool'> True
<class 'list'> ['abc', 'def']
<class 'bool'> True
<class 'int'> 1000
